In [1]:
import whisper
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
import pyttsx3
import os


/home/kyudan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# 1. Whisper 모델 로드 (ASR)
asr_model = whisper.load_model("base")


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

# 1. Llama 모델 로드
llama_model_name = "meta-llama/Llama-3.2-1B-Instruct"  # 원하는 Hugging Face 모델 경로
tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
model = AutoModelForCausalLM.from_pretrained(llama_model_name)

# 2. HuggingFace pipeline 생성
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,  # 생성할 텍스트의 최대 토큰 수
    device=0  # GPU를 사용하려면 0으로 설정, CPU를 사용하려면 -1로 설정
)

# 3. LangChain의 HuggingFacePipeline으로 Wrapping
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 이제 llm을 LangChain과 함께 사용할 수 있습니다.


2024-12-12 18:26:24.913362: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 18:26:25.048362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733995585.081293   56234 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733995585.092984   56234 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 18:26:25.239080: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:

# 3. Vector Store 세팅 (Chroma)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory="db", embedding_function=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


/tmp/ipykernel_56234/2834149021.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/tmp/ipykernel_56234/2834149021.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="db", embedding_function=embeddings)


In [5]:
from gtts import gTTS
import time
import pygame

# 4. 한국어 TTS 엔진 설정
# 7. TTS (텍스트 -> 음성 출력)
def text_to_speech_kor(text):
    tts = gTTS(text=text, lang='ko')
    filename = 'voice.mp3'
    tts.save(filename)

    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        time.sleep(0.1)

    pygame.mixer.music.unload()
    os.remove(filename)


pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:

# 5. Retrieval + QA 체인 구성
template = """
당신은 반도체 전문가이며, 사용자의 질문에 한국어로 답합니다.
아래는 관련 컨텍스트(영어)입니다. 이 컨텍스트를 기반으로 질문에 답해주세요.

컨텍스트:
{context}

질문(한국어): {question}

답변을 한국어로 해주세요.
"""
PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])


In [7]:

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},
)


In [8]:
import sounddevice as sd
from scipy.io.wavfile import write
import whisper
import os

def record_audio(file_name="input_audio.wav", duration=5, samplerate=16000):
    """음성을 녹음하고 파일로 저장"""
    print("음성을 녹음 중입니다...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()  # 녹음이 끝날 때까지 대기
    write(file_name, samplerate, audio)
    print(f"{file_name} 파일로 저장되었습니다.")
    return os.path.abspath(file_name)

def transcribe_audio(file_path):
    """Whisper를 사용하여 음성을 텍스트로 변환"""
    print("음성을 변환 중입니다...")
    model = whisper.load_model("base")  # Whisper 모델 로드
    result = model.transcribe(file_path, language="ko")
    return result["text"]


In [9]:

# 8. 전체 흐름
if __name__ == "__main__":
    # 1) 사용자 음성 입력
    print("음성을 녹음 중입니다...")
    os.system("arecord -d 5 -f cd -t wav input_audio.wav")  # 5초간 녹음 (Linux 시스템 가정)
    user_question = transcribe_audio('input_audio.wav')  # Whisper로 음성 인식
    print("사용자 질문:", user_question)

    # 2) RAG + LLM 처리
    answer = qa_chain.run(user_question)
    answer = answer.split('답변을 한국어로 해주세요.')[-1]
    print("LLM 답변:", answer)

    #3) TTS로 응답 음성 출력
    text_to_speech_kor(answer)


음성을 녹음 중입니다...


Recording WAVE 'input_audio.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo


음성을 변환 중입니다...
사용자 질문:  앤모스와 피모스의 차이가 뭐야


/tmp/ipykernel_56234/2867520911.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(user_question)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM 답변: 
아래는 앤모스와 피모스의 차이가에 대한 설명입니다.

아마도 bạn는 앤모스와 피모스를 미래의 인류에 대해 대화할 때 사용하는 two-factor authentication (2FA) 방법 중 하나입니다. 2FA는 두 가지 다른 단어를 사용하여, 사용자 identity를 xác minh하기 위해 사용됩니다. 

1.  **otp (one-time password)** : 2FA의 두 번째 단어는 OTP입니다. OTP는 6-8 자리 수자리 수를 사용하여 6자리 수자리 수를 2-3 번의 OTP를 사용하는 2FA 방법입니다. 이 방법은 사용자 identity에 대한 2FA를 제공합니다.
2.  **sms (SMS)** : 2FA의 두 번째 단어는 SMS입니다. SMS는 사용자 identity에 대한 2FA를 제공하는 2FA 방법입니다. 사용자 identity를 2FA로 xác minh하기 위해 SMS를 사용합니다. 

이 두 가지 방법은 사용자 identity를 2FA로 xác minh하기 위해 사용됩니다. 앤모스와 피모스는 2FA를 사용하여 사용자 identity를 2FA로 xác minh합니다. 

다음은 앤모스와 피모스의 차이입니다.

*   **안전한 identity management**: 앤모스와 피모스는 두 가지 다른 identity management 시스템을 사용합니다. 앤모스는 identity management system을 사용하여 사용자 identity를 관리합니다. 피모스는 identity management system을 사용하여 사용자 identity를 관리합니다.
*   **대면의 identity management**: 앤모스는 대면의 identity management 시스템을 사용하여 사용자 identity를 관리합니다. 피모스는 대면의 identity management 시스템을 사용하여 사용자 identity를 관리합니다.
*   **2FA**: 앤모스와 피모스는 2FA를 사용하여 사용자 identity를 2F

KeyboardInterrupt: 

In [15]:
answer = qa_chain.run("nmosfet의 threshold voltage를 알려줘")
answer = answer.split('답변을 한국어로 해주세요.')[-1]
print("LLM 답변:", answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM 답변: 
1. nmosfet의 threshold voltage는 0.7V입니다.
2. nmos
